In [1]:
from pringles.simulator import Simulator
mySimulator = Simulator(cdpp_bin_path='bin/', user_models_dir='src/')

In [2]:
for atomic in mySimulator.atomic_registry.discovered_atomics:
    if atomic.__name__ == "Server":
        Server = atomic
    elif atomic.__name__ == "AutoScaler":
        AutoScaler = atomic
    elif atomic.__name__ == "ServerQueue":
        ServerQueue = atomic
    elif atomic.__name__ == "Dispatcher":
        Dispatcher = atomic
    elif atomic.__name__ == "Attacker":
        Attacker = atomic

print(Server)
print(Dispatcher)
print(ServerQueue)
print(Attacker)

<class 'pringles.models.models.Server'>
<class 'pringles.models.models.Dispatcher'>
<class 'pringles.models.models.ServerQueue'>
<class 'pringles.models.models.Attacker'>


In [3]:
number_of_servers = 10


autoScalerDefaultConfig = {
    'numberOfServers': number_of_servers,
    'exponentialWeight' : 0.6,
    'loadLowerBound': 0.3,
    'loadUpperBound': 0.8,
    'loadUpdatesToBreakIdle': 10,
    'server0': 'free',
}


dispatcherDefaultConfig = {
    'numberOfServers': number_of_servers
}
for i in range(number_of_servers):
    dispatcherDefaultConfig['server' + str(i)] = 'free'
    autoScalerDefaultConfig['server' + str(i)] = 'free'


serverDefaultConfig = {
    'distribution': 'exponential',
    'mean': 0.001,
    'setupTime': '00:00:10:00'
}

serverOnDefaultConifg = {
    'initialStatus': 'free',
    **serverDefaultConfig
}

serverOffDefaultConifg = {
    'initialStatus': 'off',
    **serverDefaultConfig
}

serverQueueDefaultConfig = {
    'size': 1000,
    'currentSizeFrequency': '00:00:00:10'
}

attackerDefaultConfig = {
    'file': '../../attack-data/ataque.txt'
}



In [4]:
from pringles.models.errors import PortNotFoundException

scaler = AutoScaler('autoscaler', **autoScalerDefaultConfig)
dispatcher = Dispatcher('dispatcher', **dispatcherDefaultConfig)
queue = ServerQueue('queue', **serverQueueDefaultConfig)
servers = {}
for i in range(number_of_servers):
    server_name = 'server' + str(i)
    servers[i] = Server(server_name, **serverOnDefaultConifg)
    try:
        dispatcher.get_port(server_name)
        scaler.get_port(server_name)
    except PortNotFoundException:
        dispatcher.add_outport(server_name)
        scaler.add_outport(server_name)

attacker = Attacker('attacker', **attackerDefaultConfig)

In [5]:
print("--Scaler ports--")
print("Inport names: ", [port.name for port in scaler.inports])
print("Outport names: ", [port.name for port in scaler.outports])


print("--Dispatcher ports")
print("Inport names: ", [port.name for port in dispatcher.inports])
print("Outport names: ", [port.name for port in dispatcher.outports])

print("--Server ports--")
print("Inport names: ", [port.name for port in servers[0].inports])
print("Outport names: ", [port.name for port in servers[0].outports])

print("--ServerQueue ports--")
print("Inport names: ", [port.name for port in queue.inports])
print("Outport names: ", [port.name for port in queue.outports])

print("--Attacker ports--")
print("Inport names: ", [port.name for port in attacker.inports])
print("Outport names: ", [port.name for port in attacker.outports])

--Scaler ports--
Inport names:  ['queueLoad', 'serverResponse']
Outport names:  ['serverStatus', 'loadAvg', 'server0', 'server1', 'server2', 'server3', 'server4', 'server5', 'server6', 'server7', 'server8', 'server9']
--Dispatcher ports
Inport names:  ['newJob', 'jobDone', 'serverStatus']
Outport names:  ['requestJob', 'server0', 'server1', 'server2', 'server1', 'server2', 'server3', 'server4', 'server5', 'server6', 'server7', 'server8', 'server9']
--Server ports--
Inport names:  ['job', 'powerSignal']
Outport names:  ['done', 'ready']
--ServerQueue ports--
Inport names:  ['in', 'emit']
Outport names:  ['out', 'discarded', 'queueLoad']
--Attacker ports--
Inport names:  []
Outport names:  ['attack']


In [55]:
from pringles.models import Coupled

subcomponents = [dispatcher, queue, attacker, scaler]
for i in range(number_of_servers):
    subcomponents.append(servers[i])
    
    
top_model = Coupled(name='top', subcomponents=subcomponents)

# adding top inports
top_model.add_inport("serverStatus")
top_model.add_inport("queueLoad")
top_model.add_outport('discarded')
top_model.add_outport('jobDone')
top_model.add_outport('loadAvg')

# adding serverStatus coupling to inport of dispatcher
top_model.add_coupling('serverStatus', dispatcher.get_port("serverStatus"))

# adding couplings between queue and dispatcher
top_model.add_coupling(queue.get_port("out"), dispatcher.get_port("newJob"))
top_model.add_coupling(dispatcher.get_port("requestJob"), queue.get_port("emit"))

# adding coupling between attacker and queue
top_model.add_coupling(attacker.get_port('attack'), queue.get_port('in'))
                                                                   

# adding coupling between queue discarded port and top
top_model.add_coupling(queue.get_port('discarded'), 'discarded')

# adding coupling between queue queueLoad and scaler
top_model.add_coupling(queue.get_port('queueLoad'), 'queueLoad')


# adding couplings between dispatcher, scaler and servers
for i in range(number_of_servers):
    server_name = 'server' + str(i)
    # dispatcher/servers couplings
    top_model.add_coupling(dispatcher.get_port(server_name), servers[i].get_port('job'))
    top_model.add_coupling(servers[i].get_port('done'), dispatcher.get_port('jobDone'))
    
    # scaler/servers couplings
    top_model.add_coupling(servers[i].get_port('ready'), scaler.get_port('serverResponse'))
    top_model.add_coupling(scaler.get_port(server_name), servers[i].get_port('powerSignal'))
    
    # top/servers couplings
    top_model.add_coupling(servers[i].get_port('done'), 'jobDone')
    

# adding coupling between scaler and top of loadAvg to plot over time
top_model.add_coupling(scaler.get_port('loadAvg'), 'loadAvg')

# adding coupling between queue and scaler, where queueLoad is informed
top_model.add_coupling(queue.get_port('queueLoad'), scaler.get_port('queueLoad'))

print("Top model created")
top_model

Top model created


In [54]:
from pringles.simulator import Simulation, Event
from pringles.utils import VirtualTime

sim_evers = []


a_simulation = Simulation(top_model = top_model, 
                          duration = VirtualTime.of_minutes(1),
                          working_dir='sim_results/'
                         )

dumped_top_model_path = mySimulator.dump_model_in_file(a_simulation.top_model, a_simulation.output_dir)

# results = mySimulator.run_simulation(a_simulation)

print(dumped_top_model_path)

sim_results/2019-10-14-164329-ef71fd2ab7374b518783321a4973ad5d/top_model


In [47]:
print(results.get_process_output())


In [48]:
# print(results.get_process_error_output())

In [49]:
# display(results.output_df.head(100))

In [ ]:
print(results.logs_dfs.keys(),'\n\n')
display(results.logs_dfs['ParallelRoot'].head())

In [25]:
# Veamos que le llegan al server 0 los primneros 2 jobs, hasta que lo apagan en segundo 40
display(results.logs_dfs['server0'])

,0,1,message_type,time,model_origin,port,value,model_dest
0,0,L,X,00:00:00:000,top(09),job,0.0,server0(04)
1,0,L,X,00:00:00:000,top(09),job,1.0,server0(04)
2,0,L,X,00:00:00:000,top(09),job,2.0,server0(04)
3,0,L,X,00:00:00:000,top(09),job,3.0,server0(04)
4,0,L,X,00:00:00:000,top(09),job,4.0,server0(04)
5,0,L,X,00:00:00:000,top(09),job,5.0,server0(04)
6,0,L,X,00:00:00:000,top(09),job,6.0,server0(04)
7,0,L,X,00:00:00:000,top(09),job,7.0,server0(04)
8,0,L,X,00:00:00:000,top(09),job,8.0,server0(04)
9,0,L,X,00:00:00:000,top(09),job,9.0,server0(04)


In [26]:
# Veamos que el siguiente job le llega al server 1 en el segundo 42, y luego lo apagan en el segundo 43
# Y se espera a que termine para apagarlo
display(results.logs_dfs['server1'])

,0,1,message_type,time,model_origin,port,value,model_dest


In [27]:
# Por último cuando el 0 y el 1 están apagados, cuando llega el último job en el segundo 44, se lo envía al server 2
# que lo termina (ver logs dispatcher)
display(results.logs_dfs['server2'])

,0,1,message_type,time,model_origin,port,value,model_dest


In [ ]:
display(results.logs_dfs['dispatcher'])